# Capítulo 3: Regresión lineal

## Objetivos de aprendizaje
<hr>

- Introduction of Model-Building Strategy of a time series
- practice plot skill in python

## 1. Introducción
<hr>

## 2. Modelo estadístico
<hr>

Un modelo de regresión lineal simple parte de la hipótesis de tener una distribución normal (distribución gaussiana) para cada uno de los valores de la variable independiente, pero con una varianza constante y un error no correlacionado para todo el modelo, esto nos permite definir el modelo de dos maneras distintas que se muestran a continuación:


1. En la primera forma se modela la variable dependiente $Y$ como una función de primer orden de la variable dependiente $X$ adicionado un valor aleatorio como representación del error asociado con una distribución $\mathcal{N}(0,\sigma^{2})$.

$$Y_i=\beta_0 + \beta_1X_i + \epsilon_i,\quad \forall \quad \epsilon_i\sim \mathcal{N}(0,\sigma^{2})$$

2. Esta segunda forma se modela la variable dependiente $Y$ como una distribución normal donde la media se define como una función de primer orden, con una varianza constante.

$$Y_i \sim \mathcal{N}(\beta_0 + \beta_1X_i,\sigma^{2})$$


Independientemente de la forma de modelar este modelo la varianza siempre es constante y los parámetros del modelo tienen el mismo vector $\Theta = (\,\beta_0, \beta_1, \sigma)\,^T$, para estimar este vector se suele utilizar los métodos de:
* [Estimación por mínimos cuadrados](#Estimación-por-mínimos-cuadrados)
* [Estimación por máxima verosimilitud](#Estimación-por-máxima-verosimilitud)

La siguiente figura se busca dar claridad como un modelo de regresión lineal sigue una distribución normal con varianza constante.

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
options( warn = -1 )


linealModelPlot  = function(x, y, model)
{
    xlim = c(min(x)*0.95, max(x)*1.05)
    ylim = c(floor(min(y)*0.95), ceiling(max(y)*1.05))
    b0 = summary(model)$coefficients[1, 1]
    b1 = summary(model)$coefficients[2, 1]
    variance = summary(model)$dispersion
    sd = sqrt(variance)
    y_pred = predict(model, type="response")
    UpPred = qnorm(.95, y_pred, sd)
    LwPred = qnorm(.05, y_pred, sd)
    plotData = lapply(seq(along=x),
                      function(i)
                      {
                           stp <- 251
                           x = rep(x[i], stp)
                           y = seq(ylim[1], ylim[2], length=stp)
                           z0 = rep(0, stp)
                           z = dnorm(y, y_pred[i], sd)
                           return(list(x=x, y=y, z0=z0, z=z))
                      }
                        )
    par(mfrow=c(1,1))
    n = 2
    N = length(y_pred)
    zMax = max(unlist(sapply(plotData, "[[", "z")))*1.5
    mat = persp(xlim, ylim, matrix(0, n, n), zlim=c(0, zMax), theta=-30, ticktype="detailed",box=FALSE)
    C = trans3d(x, UpPred, rep(0, N),mat)
    lines(C, lty=2)
    C = trans3d(x, LwPred, rep(0, N), mat)
    lines(C, lty=2)
    C = trans3d(c(x, rev(x)), c(UpPred, rev(LwPred)), rep(0, 2*N), mat)
    polygon(C, border=NA, col=adjustcolor("yellow", alpha.f = 0.5))
    C = trans3d(x, y_pred, rep(0, N), mat)
    lines(C, lwd=2, col="grey")
    C = trans3d(x, y, rep(0,N), mat)
    points(C, lwd=2, col="#00526D")
    for(j in N:1)
    {
        xp = plotData[[j]]$x
        yp = plotData[[j]]$y
        z0 = plotData[[j]]$z0
        zp = plotData[[j]]$z
        C = trans3d(c(xp, xp), c(yp, rev(yp)), c(zp, z0), mat)
        polygon(C, border=NA, col="light blue", density=40)
        C = trans3d(xp, yp, z0, mat)
        lines(C, lty=2)
        C = trans3d(xp, yp, zp, mat)
        lines(C, col=adjustcolor("blue", alpha.f = 0.5))
    }
}



icecream = data.frame(temp=c(11.9, 11.9, 15.2, 16.4, 17.2, 18.1, 18.5, 19.4, 22.1, 22.6, 23.4, 25.1),
                      units=c(185, 215, 332, 325, 408, 421, 406, 412, 522, 445, 544, 614)
                     )
market.size = 800
icecream$opportunity = market.size - icecream$units

model = glm(units ~ temp, data=icecream, family=gaussian(link="identity"))
linealModelPlot(x = icecream$temp, y=icecream$units, model=model)

##  Apéndice
<hr>

### Estimación por mínimos cuadrados 

<hr>

La estimación de los mínimos cuadrados se calcula utilizando un ajuste de regresión de una línea recta buscando la optimización del menor error cuadrático medio.


\begin{align*} 
\mathcal{s}(\,\,\beta_0, \beta_1)\,&= \sum_{i=1}^{n}{\epsilon_i}\\
&=\sum_{i=1}^n(y_i-\beta_0-\beta_1x_i)^2 
\end{align*}

Derivando e igualando a cero, es decir,

\begin{align*} 
\frac{\partial S(\beta_0, \beta_1)}{\partial \beta_0}&=0\\
\frac{\partial S(\beta_0, \beta_1)}{\partial \beta_1}&=0
\end{align*}

obtenemos que los estimadores para $\beta_0$ y $\beta_1$:<br>
\begin{gather*} 
\widehat{\beta}_0=\overline{y}-\widehat{\beta}_1\overline{x}\quad \text{y}\quad \widehat{\beta}_1=\frac{S_{xy}}{S_{xx}}
\end{gather*}

donde:

\begin{gather*} 
S_{xx}=\sum_{i=1}^n(x_i-\overline{x})^2\\
S_{yy}=\sum_{i=1}^n(y_i-\overline{y})^2\\
S_{xy}=S_{yx}\\
S_{yx}=\sum_{i=1}^n(x_i-\overline{x})(y_i-\overline{y})
\end{gather*}


### Estimación por máxima verosimilitud

<hr>

El método más común para estimar parámetros es el método de máxima verosimilitud de un modelo donde $X_1,...,X_n$ independientes e idénticamente distribuidas con función de densidad de probabilidad $P(x,\theta)$ entonces se define:

* función de verosimilitud:
\begin{gather*} 
\mathcal{L}(\theta) = \prod_{i=1}^n p(x_i;\theta).
\end{gather*}

* función de log-verosimilitud
\begin{gather*} 
\mathcal{l}(\theta)=\ln \mathcal{L}(\theta)=\sum_{i=1}^n \ln p(x_i; \theta)
\end{gather*}

por ende, el estimador de máxima verosimilitud simplemente es el valor   $\theta $  que maximiza $\mathcal{L}(\theta)$.En el caso particular de un modelo de regresión lineal simple parte de la hipótesis de tener una distribución normal por consíguete su función de densidad se define como:

\begin{gather*}
f(x)=\frac{1}{\sigma\sqrt{2\pi}}\exp\left\{-\frac{(x-\overline{x})^2}{2\sigma^2}\right\}
\end{gather*}

Al remplazar la función de densidad en la ecuación de log-verosimilitud. Se obtiene la siguiente expresión:

\begin{align*}
\mathcal{l}(\theta)=&\sum_{i=1}^n \ln\left(\frac{1}{\sigma\sqrt{2\pi}}\exp\left\{-\frac{(Y_i-\beta_0-\beta_1X_i)^2}{2\sigma^2}\right\}\right)\\
\mathcal{l}(\theta)=&-\left(\frac{n}{2}\right)\ln(2\pi)-\left(\frac{n}{2}\right)\ln\left(\sigma^2\right)-\left(\frac{1}{2\sigma^2}\right)\sum_{i=1}^n (Y_i - \beta_0 - \beta_1 X_i)^2
\end{align*}

erivando parcialmente con respecto a $\beta_0$, $\beta_1$ y $\sigma^2$ e igualando a cero, obtenemos los estimadores de máxima verosimilitud:

\begin{align*}
\widetilde{\beta}_0 =&\overline{Y}-\widehat{\beta}_1\overline{X} \\
\widetilde{\beta}_1 =&\frac{S_{xy}}{S_{xx}}\\
\widetilde{\sigma}^2=&\frac{\sum_{i=1}^n(Y_i-\widetilde{\beta}_0-\widetilde{\beta}_1X_i)^2}{n}
\end{align*}